In [7]:
pip install textblob nltk


In [10]:
import pandas as pd
from textblob import TextBlob

df = pd.read_csv("/content/Facebook-datasets.csv")

df = df.dropna(subset=['comment_text'])

def get_sentiment(text):
    analysis = TextBlob(str(text))
    polarity = analysis.sentiment.polarity
    if polarity > 0:
        return 'positive'
    elif polarity < 0:
        return 'negative'
    else:
        return 'neutral'


df['sentiment'] = df['comment_text'].apply(get_sentiment)

df.to_csv("/Facebook-dataset-with-sentiment.csv", index=False)#here used text blob because csv file doesnot contain the column if the comment is P,N,N


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import nltk
from nltk.corpus import stopwords
import string
import re



In [12]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

df = pd.read_csv("/Facebook-dataset-with-sentiment.csv")



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:

text_col = 'comment_text'
label_col = 'sentiment'

In [14]:

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = ''.join([char for char in text if char not in string.punctuation])
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

In [15]:
df[text_col] = df[text_col].astype(str).apply(preprocess_text)

X_train, X_test, y_train, y_test = train_test_split(df[text_col], df[label_col], test_size=0.2, random_state=42)

In [16]:


vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [17]:

model = LogisticRegression(max_iter=200)
model.fit(X_train_vec, y_train)

LogisticRegression(max_iter=200)

In [18]:


y_pred = model.predict(X_test_vec)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7150259067357513

Classification Report:
               precision    recall  f1-score   support

    negative       0.61      0.37      0.46        38
     neutral       0.71      0.91      0.80        90
    positive       0.78      0.65      0.71        65

    accuracy                           0.72       193
   macro avg       0.70      0.64      0.65       193
weighted avg       0.71      0.72      0.70       193



In [20]:
while True:
    user_input = input("\nEnter a sentence (or type 'exit' to stop): ")
    if user_input.lower() == 'exit':
        break

    cleaned_input = preprocess_text(user_input)
    input_vec = vectorizer.transform([cleaned_input])
    prediction = model.predict(input_vec)[0]
    print(f"Predicted Sentiment: {prediction}")



Enter a sentence (or type 'exit' to stop): I'm really happy with the service, everything was perfect!
Predicted Sentiment: positive

Enter a sentence (or type 'exit' to stop): I feel so lonely and disappointed; nothing seems to go right.
Predicted Sentiment: positive

Enter a sentence (or type 'exit' to stop): I went to the store to buy some groceries and came back home.
Predicted Sentiment: neutral

Enter a sentence (or type 'exit' to stop): i am feeling alone i am going to die!
Predicted Sentiment: neutral

Enter a sentence (or type 'exit' to stop): I’ve been feeling empty and broken inside, and no one seems to understand.
Predicted Sentiment: neutral

Enter a sentence (or type 'exit' to stop): i am very happy with the service
Predicted Sentiment: positive

Enter a sentence (or type 'exit' to stop): exit
